> ### Note on Labs and Assignments:
>
> 🔧 Look for the **wrench emoji** 🔧 — it highlights where you're expected to take action!
>
> These sections are graded and are not optional.
>

# IS 4487 Lab 14: API Integration

## Outline
1. Import customer reviews
2. Create prompts for LLM
3. Summarize Customer Reviews

<a href="https://colab.research.google.com/github/Stan-Pugsley/is_4487_base/blob/main/Labs/lab_14_API.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Data Description

We will use a dataset containing sample customer reviews for Megatelco.  Each review contains a date, a number of stars, and the review text.

Megatelco management will use the reviews to direct their efforts to improve customer retention.

| Column                        | Data Type       | Description                                                  |
|------------------------------|------------------|--------------------------------------------------------------|
| `Date`                   | Date           | Date of the review                                              |
| `Stars`                 | Integer           | Number of stars, from 1 (low) to 5 (high)                                     |
| `Review`             | String       | Text of the customer review                      |


# Business Objective

Your goal is to summarize the customer feedback for management.  They will be looking for the themes, sentiment and action items to address customer churn.


## Part 1: Load the Data

### What you are going to do:
- Load the dataset
- Preview the data

**Things to notice:**
- Do you see any elements in the reviews that would difficult for VADER or other lexicon-based models to process?


In [1]:
import pandas as pd
import google.generativeai as genai

Create a dataframe containing 10 sample reviews for use in the lab

In [2]:
reviews_data = {
    "Date": [
        "2025-12-01",
        "2025-12-03",
        "2025-12-05",
        "2025-12-08",
        "2025-12-10",
        "2025-12-12",
        "2025-12-15",
        "2025-12-18",
        "2025-12-20",
        "2025-12-22",
    ],
    "Stars": [4, 3, 5, 4, 2, 3, 3, 4, 1, 5],
    "Review": [
        "I purchased a Megatelco phone last week, and it has sick performance. The camera quality is great, and the battery life is long. Overall, a solid 4-star experience.",
        "My Megatelco phone arrived fast, but I've noticed some issues with the software. It tends to lag at times, affecting the user experience. Customer support was helpful, but I expected better performance. 3 stars.",
        "Megatelco offers top-notch phones! I'm happy with the features and the smooth interface. The camera takes stunning photos, and the battery lasts all day. 5 stars without a doubt!",
        "I've been using Megatelco phones for a while now, and they never disappoint. The latest model I purchased is no exception. Great performance, stylish design, and excellent customer service. 4 stars from a satisfied customer.",
        "Regrettably, my experience with Megatelco was not as expected. The phone had constant connectivity issues, and the support team couldn't provide a solution. Disappointed and only giving it 2 stars.",
        "Megatelco's latest phone is so so.  It has some good features but is overpriced. The customer service team was quick to address a minor concern I had. I'll give it a 3-star rating",
        "I had a mixed experience with Megatelco. The phone's performance is decent, but I encountered some software glitches. Customer support was responsive but couldn't entirely resolve the issue. 3 stars.",
        "Megatelco phones are reliable and have good prices. I especially like the new iPhone model. I'm giving it 4 stars for its overall quality.",
        "My Megatelco phone started malfunctioning within a week of purchase. The touch screen wouldn't respond, and customer support was unable to provide a quick solution. Unfortunately, a 1-star rating.",
        "I recently upgraded to the latest Megatelco phone, and it's fantastic! The speed, camera quality, and battery life are exceptional. The sleek design is a bonus. Easy 5 stars!",
    ],
}

df = pd.DataFrame(reviews_data)
df.head()

,Date,Stars,Review
0,2025-12-01,4,"I purchased a Megatelco phone last week, and i..."
1,2025-12-03,3,"My Megatelco phone arrived fast, but I've noti..."
2,2025-12-05,5,Megatelco offers top-notch phones! I'm happy w...
3,2025-12-08,4,I've been using Megatelco phones for a while n...
4,2025-12-10,2,"Regrettably, my experience with Megatelco was ..."


# Part 2 : Prepare the LLM Prompt
### What you are going to do:
- Instruct the LLM on the context and desired output

### Why this matters:
A LLM needs to know what you are trying to accomplish, what data you will provide, what it should do with the data, and how to format the output.  Your prompt needs to set all of this context before passing in a review.  

In [3]:
base_prompt = (
    "Summarize the sentiment and most important points in the following user review for "
    "a phone company named Megatelco.  You will evaluate the sentiment, theme (in 2 words or less), count the number of words, and suggest an action (in 2 words or less)."
     "Format the output in a table with columns: "
     "Sentiment, Theme, Word Count, Suggested Action. Review: "
)

#Get the first review and pair it with the prompt
review = df['Review'].values[0]
prompt = base_prompt + review
print(prompt)

Summarize the sentiment and most important points in the following user review for a phone company named Megatelco.  You will evaluate the sentiment, theme (in 2 words or less), count the number of words, and suggest an action (in 2 words or less).Format the output in a table with columns: Sentiment, Theme, Word Count, Suggested Action. Review: I purchased a Megatelco phone last week, and it has sick performance. The camera quality is great, and the battery life is long. Overall, a solid 4-star experience.


### 🔧 Try It Yourself — Part 2

1. Create a new version of the prompt that adds two additional columns to the output.   (this columns should be numeric values you can visualize at a later time)

### In Your Response:
1. Why did you pick the two columns that you added?  What business insight would they provide?

In [4]:
# 🔧 Enter your code here
base_prompt = (
    "Summarize the sentiment and most important points in the following user review for "
    "a phone company named Megatelco.  You will evaluate the sentiment, theme (in 2 words or less), count the number of words, suggest an action (in 2 words or less), the star rating given in the review, and the character count of the review."
    "Format the output in a table with columns: "
    "Sentiment, Theme, Word Count, Suggested Action, Star Rating, Character Count. Review: "
)

#Get the first review and its star rating and calculate character count
review = df['Review'].values[0]
star_rating = df['Stars'].values[0]
character_count = len(review)

# Construct the prompt with the review, star rating, and character count
prompt = base_prompt + f"{{review}} Star Rating: {{star_rating}} Character Count: {{character_count}}"
print(prompt)

Summarize the sentiment and most important points in the following user review for a phone company named Megatelco.  You will evaluate the sentiment, theme (in 2 words or less), count the number of words, suggest an action (in 2 words or less), the star rating given in the review, and the character count of the review.Format the output in a table with columns: Sentiment, Theme, Word Count, Suggested Action, Star Rating, Character Count. Review: {review} Star Rating: {star_rating} Character Count: {character_count}


### ✍️ Your Response: 🔧
1.

# Part 3: Connect with the API and Test

### What you are going to do:
- Create a connection to Gemini
- Run a test prompt
- Pass the full collection of reviews to the API (either in a batch or one-by-one in a loop)
- Format the output in a dataframe.   

### Do the following
- Go to https://aistudio.google.com/api-keys
- Click on the `Get API key` link on the bottom left corner
- Copy the value into the box below
- Send the first customer review to Gemini for analysis, then view the result

**Things to notice:**
- Is there any limit to the number of free requests you can make to Gemini?  (without payment)

In [ ]:
# Configure the API key
API_KEY = '##Paste your API key here##'
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel('models/gemini-2.5-flash')
print("Gemini API configured successfully.")

In [ ]:
response = model.generate_content(prompt)
print(response.candidates[0].content.parts[0].text)

### 🔧 Try It Yourself — Part 3
Ask Gemini to help you loop through the reviews, one by one, and format them into a dataframe. Use the following steps:
1. Build a full prompt by combining the base prompt that you created above with one or more reviews
2. Pass the full prompt to Gemini
3. Format the response into a dataframe
4. If you are processing one row at a time, pass the next prompt (in a loop) until you have processed all 10 reviews
5. Show the final dataframe using `df.head()`

### In Your Response:
1. How does the output of the LLM compare to the output we saw in week 13 from VADER or TextBlob?

In [ ]:
# 🔧 Add code here

### ✍️ Your Response: 🔧
1.



# Part 4: Visualize the Output
### What you are going to do:
- create visualizations to summarize the customer reviews.

## Why this matters:
If we have thousands of reviews, you will need to summarize them for management use.  Each chart should tell a distinct story about the customer feedback, themes and suggested action items.  

### 🔧 Try It Yourself — Part 4
Create at least four visualizations to answer the following questions:
1. What are the main themes?
2. For each theme, what is the sentiment associated with the theme?
3. What are the action items that should be taken to reduce churn?
4. Add one or more visualizations that will show the insights from the fields that you added in part 2.  

### In Your Response:
1. Why did you pick the charts or image types for each of the four visualizations?  

In [ ]:
# 🔧 Enter your code here

### ✍️ Your Response: 🔧
1.


## 🔧 Part 5: Reflection (100 words or less)

In this lab you connected to an LLM API to request summarization of customer reviews.  

Use the cell below to answer the following questions:

1. What was the elapsed time to collect the LLM responses to all 10 requests?  How long would it take to process 1,000 requests?
2. What are the advantages and disadvantes of using Gemini versus VADER or TextBlob, which we used in Lab 13?  
3. Write a prompt that you could use to an LLM to create a business strategy and business plan to improve customer churn.   

### ✍️ Your Response: 🔧
1.

2.

3.

# Export Your Notebook to Submit in Canvas
Use the instructions from Lab 1

In [ ]:
!jupyter nbconvert --to html "lab_14_LastnameFirstname.ipynb"